In [ ]:
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
import visualizer_helpers as vh

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

## Trip Purpose

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## Trip Destination

In [ ]:
base_trip = pd.read_csv(f"{base_dir}/final_trips.csv", 
                        index_col='trip_id', 
                        usecols=['trip_id','trip_count','trip_num','dtaz'])

build_trip = pd.read_csv(f"{build_dir}/final_trips.csv", 
                         index_col='trip_id', 
                         usecols=['trip_id','trip_count','trip_num','dtaz'])


In [ ]:
df = base_trip[base_trip.trip_count != base_trip.trip_num].merge(build_trip,
                                                                 how='left',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))

df = (df.dtaz_base == df.dtaz_build).value_counts()
df.index = df.index.map({True:'No Change',False:'Change'})
df

In [ ]:
fig = vh.create_pie_chart(df.to_frame(),["count"])
fig.show()

## Trip Scheduling

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## Trip Mode Choice

In [ ]:
print("Lorem ipsum") # Placeholder for CS